In [1]:
'''
Date: 25 Aug 2020

Python version:      3.7
PyTorch version:     1.2.0

@author: Maksim Lavrov
Modified by Linnea Evanson / changed from Vanilla baseline to flexBaseline

CIFAR10 dataset

VGG16 baseline model
'''


 #===================================================== Import libraries ================================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn 
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# ================================================= Base VGG-16 Network ================================================================================
# ================================================= Flexible Layer ================================================================================

    
class FlexiLayer_base(nn.Module): # class FlexiLayer(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(FlexiLayer_base, self).__init__()
        
        self.t_1 = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.t_2 = nn.MaxPool2d(self.kernel_size, self.stride, self.padding) # get max result with the same kernel size
        self.m = nn.Sigmoid()
        
        self.threshold1 = Variable(torch.randn((1, self.out_channels, 30, 30)))
        
        self.thresh_mean = []
        
    def forward(self, t):
        
        #self.threshold1.expand(t.size(0), self.out_channels, 30, 30)
        
        
        #cond = torch.sub(self.t_2(t), self.threshold1.cuda())
        #t_2_2 = self.m(cond*50)*self.t_2(t) # 
        #t_1_1 = self.m(cond*(-50))*self.t_1(t) # 
        #t = torch.add(t_2_2, t_1_1)
        
        return self.t_1(t)

    # ================================================= VGG-16 Network ================================================================================
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()

        self.block1 = nn.Sequential(
                      nn.Conv2d(in_channels = 3,out_channels = 64,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(64),
                      nn.ReLU(),
                      FlexiLayer_base(in_channels = 64,out_channels = 64,kernel_size = 3, padding =0),
                      nn.BatchNorm2d(64),
                      nn.ReLU(),
                      #nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.3))

        self.block2 = nn.Sequential(
                      nn.Conv2d(in_channels = 64,out_channels = 128,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(128),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 128,out_channels = 128,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(128),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.4))

        self.block3 = nn.Sequential(
                      nn.Conv2d(in_channels = 128,out_channels = 256,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 256,out_channels = 256,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(256),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.4))

        self.block4 = nn.Sequential(
                      nn.Conv2d(in_channels = 256,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2) ,
                      nn.Dropout2d(0.4))

        self.block5 = nn.Sequential(
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3,padding = 1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.Conv2d(in_channels = 512,out_channels = 512,kernel_size = 3, padding =1),
                      nn.BatchNorm2d(512),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Dropout2d(0.5) )

        self.fc =     nn.Sequential(
                      nn.Linear(512,100),
                      nn.Dropout(0.5),
                      nn.BatchNorm1d(100),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(100,10), )

    def forward(self,x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)

        return out



In [2]:


from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms

#from models.VGG16 import *

epsilons = [0, .05, .1, .15, .2, .25, .3]

# ======================================== prepare the dataset ==========================================================================================
mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 100

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])


trainset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=True, download= True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)



Files already downloaded and verified
Files already downloaded and verified


In [3]:
from __future__ import print_function

import json
import os
import sys

import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from PyHessian.utils import *
from PyHessian.density_plot import get_esd_plot
from PyHessian.density_plot import density_generate

from PyHessian.models.resnet import resnet
from PyHessian.pyhessian import hessian



def get_eigenvalues(model, trainloader, testloader):

    #*
    # @file Different utility functions
    # Copyright (c) Zhewei Yao, Amir Gholami
    # All rights reserved.
    # This file is part of PyHessian library.
    #
    # PyHessian is free software: you can redistribute it and/or modify
    # it under the terms of the GNU General Public License as published by
    # the Free Software Foundation, either version 3 of the License, or
    # (at your option) any later version.
    #
    # PyHessian is distributed in the hope that it will be useful,
    # but WITHOUT ANY WARRANTY; without even the implied warranty of
    # MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    # GNU General Public License for more details.
    #
    # You should have received a copy of the GNU General Public License
    # along with PyHessian.  If not, see <http://www.gnu.org/licenses/>.
    #*

    # Settings
    parser = argparse.ArgumentParser(description='PyTorch Example')
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
    parser.add_argument(
        '--mini-hessian-batch-size',
        type=int,
        default=200,
        help='input batch size for mini-hessian batch (default: 200)')
    parser.add_argument('--hessian-batch-size',
                        type=int,
                        default=200,
                        help='input batch size for hessian (default: 200)')
    parser.add_argument('--seed',
                        type=int,
                        default=1,
                        help='random seed (default: 1)')
    parser.add_argument('--batch-norm',
                        action='store_false',
                        help='do we need batch norm or not')
    parser.add_argument('--residual',
                        action='store_false',
                        help='do we need residual connect or not')

    parser.add_argument('--cuda',
                        action='store_false',
                        help='do we use gpu or not')
    parser.add_argument('--resume',
                        type=str,
                        default='fmnist_flex_model_14_52.pth',
                        help='get the checkpoint')

    args = parser.parse_args(args=[])
    # set random seed to reproduce the work
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    for arg in vars(args):
        print(arg, getattr(args, arg))

    # get dataset
    #train_loader, test_loader = getData(name='cifar10_without_dataaugmentation',
    #                                    train_bs=args.mini_hessian_batch_size,
    #                                    test_bs=1)

    train_loader, test_loader = testloader, trainloader

    ##############
    # Get the hessian data
    ##############
    assert (args.hessian_batch_size % args.mini_hessian_batch_size == 0)
    assert (50000 % args.hessian_batch_size == 0)
    batch_num = args.hessian_batch_size // args.mini_hessian_batch_size

    if batch_num == 1:
        for inputs, labels in train_loader:
            hessian_dataloader = (inputs, labels)
            break
    else:
        hessian_dataloader = []
        for i, (inputs, labels) in enumerate(train_loader):
            hessian_dataloader.append((inputs, labels))
            if i == batch_num - 1:
                break

    if args.cuda:
        model = model.cuda()
    model = torch.nn.DataParallel(model)

    criterion = nn.CrossEntropyLoss()  # label loss

    ###################
    # Get model checkpoint, get saving folder
    ###################
    if args.resume == '':
        raise Exception("please choose the trained model")
    #model.load_state_dict(torch.load(args.resume))
    #state = torch.load('fmnist_base_model_14_28.pth')
    #model.load_state_dict(state['model'])

    ######################################################
    # Begin the computation
    ######################################################

    # turn model to eval mode
    model.eval()
    if batch_num == 1:
        hessian_comp = hessian(model,
                               criterion,
                               data=hessian_dataloader,
                               cuda=args.cuda)
    else:
        hessian_comp = hessian(model,
                               criterion,
                               dataloader=hessian_dataloader,
                               cuda=args.cuda)

    print(
        '********** finish data londing and begin Hessian computation **********')
    # @Linnea: adding print test
    #print_test = hessian_comp.dataloader_hv_product(hessian_comp, None)

    #top_eigenvalues, _ = hessian_comp.eigenvalues(top_n = 100)   #get top 100 eigenvalues
    density_eigen, density_weight = hessian_comp.density(iter = 100) #get top 150 eigenvalues in faster way than line above


    return density_eigen, density_weight

In [4]:
def get_alpha(plot_height, eign_grids):

    total_values = np.sum(plot_height)

    neg_values = []
    i = 0
    for value in eign_grids:
        if value < 0:
            neg_values.append(plot_height[i])
        i += 1

    alpha  = np.sum(neg_values) /total_values

    return alpha

In [5]:
##====================Main========================
import pickle
import os

all_eigenvalues = []
all_weights = []
all_alpha = []
count = 0

#base_models = os.listdir("../Imperial_Flexible_CNNs/models3")

path = "../Imperial_Flexible_CNNs_Original/models/base_downsampled/"

for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
#    if (True): #calc only every 10 epochs (should take roughly 4.5 hours)
        print("CUDA Available: ",torch.cuda.is_available())
        use_cuda=True
        device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

        # Initialize the network
        model = VGG16().to(device)

        # Load the pretrained model
        state = torch.load(path + file)
        model.load_state_dict(state['model'])

        # Set the model in evaluation mode. In this case this is for the Dropout layers
        model.eval()
                
        density_eigen, density_weight = get_eigenvalues(model, trainloader, testloader)
        
        print("shape of density eigen:", len(density_eigen), len(density_eigen[0]))
        
        temp = file.split("_") #get the epoch number to name the photo 
        
        get_esd_plot(density_eigen, density_weight, "base_image_50downsampled_"+ str(temp[2]) + ".pdf")

        
        #plot_height, eign_grids = density_generate(density_eigen, density_weight)

        
        #all_eigenvalues.append(eign_grids)
        #all_weights.append(plot_height)

        #all_alpha.append( get_alpha(plot_height, eign_grids) )
    
    print(100*count/152, "% complete")
    count += 1
    
#Save all eigenvalues:
#with open('all_eigenvalues_flex50downsampled_fastmethod22', 'wb') as f:
#    pickle.dump(all_eigenvalues, f)

#Save the weights:
#with open('all_eignweights_flex50downsampled_fastmethod22', 'wb') as f:
#    pickle.dump(all_weights, f)
    
#Save all alpha:
#with open('all_alpha_flex50downsampled_fastmethod22', 'wb') as f:
#    pickle.dump(all_alpha, f)
    

CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)
0.0 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)
0.6578947368421053 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of

/home/levanson/NAS/Flexible_CNNs/Imperial_Flexible_CNNs/PyHessian/density_plot.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


13.157894736842104 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)


/home/levanson/NAS/Flexible_CNNs/Imperial_Flexible_CNNs/PyHessian/density_plot.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


13.81578947368421 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)


...

/home/levanson/NAS/Flexible_CNNs/Imperial_Flexible_CNNs/PyHessian/density_plot.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


98.6842105263158 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)


/home/levanson/NAS/Flexible_CNNs/Imperial_Flexible_CNNs/PyHessian/density_plot.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


99.34210526315789 % complete
CUDA Available:  True
fff 1
mini_hessian_batch_size 200
hessian_batch_size 200
seed 1
batch_norm True
residual True
cuda True
resume fmnist_flex_model_14_52.pth
********** finish data londing and begin Hessian computation **********
shape of density eigen: 1 100
lenth into get_esd_plot: 1 100
lenth into density_generate: 1 100
len into semilogy: (10000,)
100.0 % complete


/home/levanson/NAS/Flexible_CNNs/Imperial_Flexible_CNNs/PyHessian/density_plot.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [6]:
import pickle
import pandas as pd
#Load Alpha:
with open('all_alpha_flex50downsampled_fastmethod2', 'rb') as f:
    all_alpha = pickle.load(f)

#Load epsilons:

df_temp = pd.read_csv("flex_50downsampled_trainaccuracy.csv")
epsilons = df_temp['train_accuracy']
testepsilons = df_temp['test_accuracy']    
    
    
#Order alpha of Flex network 3 epochs file:

import os 
import pandas as pd
#flex_models = os.listdir("../Imperial_Flexible_CNNs/models3")
flex_models = []
epoch_number = []
count = 0
path = "../Imperial_Flexible_CNNs_Original/models/flex_downsampled/"

for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
        temp = file.split("_") #calc only every 10 epochs (should take roughly 4.5 hours)
        flex_models.append(temp)
        epoch_number.append(int(temp[2]))
        count += 1
    
#Need to parse based on first number:

#Create database:
df_flex = pd.DataFrame(list(zip(flex_models,epoch_number)), columns = ['models','epoch'])


df_flex['alpha'] = all_alpha

#print(df)

#Order dataframe by epoch number:

df_flex = df_flex.sort_values(by = ['epoch'])

display(df_flex[0:50]) 
    


,models,epoch,alpha
20,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",0,0.509046
53,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",1,0.508543
133,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",2,0.451918
44,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",3,0.472522
89,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",4,0.580286
130,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",5,0.531148
41,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",6,0.595373
18,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",7,0.412752
81,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",8,0.469765
92,"[VGG16-flex-v5-block1-nnmodule, 50xdownsampled...",9,0.490278


In [7]:
#Plot alpha - epsilon
import matplotlib.pyplot as plt
plt.plot(df_flex['alpha'],epsilons, '.', color = 'orange')
plt.title("Alpha-Epsilon Plot of Flex Network")
plt.xlabel("Alpha")
plt.ylabel("Epsilon (Train Error %)")
plt.show()

In [8]:
#Plot alpha - epsilon
import matplotlib.pyplot as plt
plt.plot(df_flex['alpha'],testepsilons, '.')
plt.title("Alpha-Epsilon Plot of Flex Network")
plt.xlabel("Alpha")
plt.ylabel("Epsilon (Test Accuracy %)")
plt.show()

In [9]:
import matplotlib.pyplot as plt
plt.plot(df_flex['epoch'],epsilons, '.')
plt.title("Epsilon-Epoch Plot of Flex Network")
plt.xlabel("Epoch")
plt.ylabel("Epsilon (Train Error %)")
plt.show()

In [10]:
import matplotlib.pyplot as plt
plt.plot(df_flex['epoch'],df_flex['alpha'], '.', color = 'orange')
plt.title("Alpha-Epoch Plot of Flex Network")
plt.xlabel("Epoch")
plt.ylabel("Alpha")
plt.show()

In [11]:
import matplotlib.pyplot as plt
plt.plot(df_flex['epoch'],df_flex['alpha'], color = 'orange')
plt.title("Alpha-Epoch Plot of Flex Network")
plt.xlabel("Epoch")
plt.ylabel("Alpha")
plt.show()